In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

import glob
import torch 

from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory
import nvtabular as nvt

/home/ec2-user/SageMaker/.persisted_conda/transformers4rec-23.04/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/ec2-user/SageMaker/.persisted_conda/transformers4rec-23.04/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")
/home/ec2-user/SageMaker/.persisted_conda/transformers4rec-23.04/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import cudf

In [3]:
# workflow = nvt.Workflow.load('./preprocessWorkflow')


In [5]:
month1 = "202201"
month3 = "202203"
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/home/ec2-user/SageMaker/data/")

mydata = nvt.Dataset(os.path.join(INPUT_DATA_DIR, "202201-202203/data.parquet/part_0.parquet"))
# dataset = nvt.Dataset(mydata)

In [ ]:
# procesed = workflow.transform(dataset)

In [6]:
mydata.head()

,recipient,timestamp-first,buyAsset-list,et_dayofweek_sin-list,txFee_eth_log_norm-list,buyQty1_log_norm-list,buyPrice_log_norm-list,token_category-list,token_rank_category-list,risky_flags-list
0,0x000000000004d7463d0f9c77383600bc82d612f5,2022-01-01 20:14:35,"[489, 418, 956, 24, 359, 1476, 1017, 714, 503,...","[0.43388293040961884, 0.43388293040961884, -0....","[-0.9175754, -0.83067816, -0.83143824, -0.8746...","[-0.025127048, -0.025127048, -0.025127048, -0....","[-0.0009817483, -0.0009817483, -0.0009817483, ...","[20, 28, 34, 4, 20, 13, 23, 13, 15, 4, 6, 4, 1...","[3, 3, 5, 3, 3, 3, 5, 5, 6, 3, 5, 3, 3, 5, 5, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, ..."
1,0x0000000000086013192a2c9a506742a43c916e1e,2022-01-09 07:44:40,"[4, 12, 150, 4]","[8.975979006501142e-07, 0.7818320421108522, 0....","[2.0464702, 0.92810464, 0.78188777, 5.520375]","[-0.025127048, -0.025127048, -0.025127048, -0....","[-0.0009817483, -0.0009817483, -0.0009817483, ...","[5, 4, 4, 5]","[3, 3, 5, 3]","[3, 3, 3, 3]"
2,0x000000000008e4db6a6194c6957df47e30970dc2,2022-02-23 18:48:34,"[4, 5, 113]","[0.43388293040961884, 0.43388293040961884, -0....","[1.1193919, -0.6704848, -0.7880669]","[-0.025127048, -0.025127048, -0.025127048]","[-0.0009817483, -0.0009817483, -0.0009817483]","[5, 3, 5]","[3, 4, 5]","[3, 4, 3]"
3,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,2022-01-28 21:47:38,"[614, 4]","[-0.9749281119157542, 8.975979006501142e-07]","[0.17984277, 0.0010065734]","[-0.025127048, -0.025127048]","[-0.0009817483, -0.0009817483]","[4, 5]","[5, 3]","[3, 3]"
4,0x00000000005ef87f8ca7014309ece7260bbcdaeb,2022-01-10 19:19:47,"[12, 3]","[0.7818320421108522, -0.7818309228245777]","[1.4960439, 1.3195738]","[-0.025127048, -0.025127048]","[-0.0009817483, -0.0009817483]","[4, 3]","[3, 4]","[3, 3]"


In [7]:
cudf_df = mydata.to_ddf().compute()


In [8]:
cudf_df.head()

,recipient,timestamp-first,buyAsset-list,et_dayofweek_sin-list,txFee_eth_log_norm-list,buyQty1_log_norm-list,buyPrice_log_norm-list,token_category-list,token_rank_category-list,risky_flags-list
0,0x000000000004d7463d0f9c77383600bc82d612f5,2022-01-01 20:14:35,"[489, 418, 956, 24, 359, 1476, 1017, 714, 503,...","[0.43388293040961884, 0.43388293040961884, -0....","[-0.9175754, -0.83067816, -0.83143824, -0.8746...","[-0.025127048, -0.025127048, -0.025127048, -0....","[-0.0009817483, -0.0009817483, -0.0009817483, ...","[20, 28, 34, 4, 20, 13, 23, 13, 15, 4, 6, 4, 1...","[3, 3, 5, 3, 3, 3, 5, 5, 6, 3, 5, 3, 3, 5, 5, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, ..."
1,0x0000000000086013192a2c9a506742a43c916e1e,2022-01-09 07:44:40,"[4, 12, 150, 4]","[8.975979006501142e-07, 0.7818320421108522, 0....","[2.0464702, 0.92810464, 0.78188777, 5.520375]","[-0.025127048, -0.025127048, -0.025127048, -0....","[-0.0009817483, -0.0009817483, -0.0009817483, ...","[5, 4, 4, 5]","[3, 3, 5, 3]","[3, 3, 3, 3]"
2,0x000000000008e4db6a6194c6957df47e30970dc2,2022-02-23 18:48:34,"[4, 5, 113]","[0.43388293040961884, 0.43388293040961884, -0....","[1.1193919, -0.6704848, -0.7880669]","[-0.025127048, -0.025127048, -0.025127048]","[-0.0009817483, -0.0009817483, -0.0009817483]","[5, 3, 5]","[3, 4, 5]","[3, 4, 3]"
3,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,2022-01-28 21:47:38,"[614, 4]","[-0.9749281119157542, 8.975979006501142e-07]","[0.17984277, 0.0010065734]","[-0.025127048, -0.025127048]","[-0.0009817483, -0.0009817483]","[4, 5]","[5, 3]","[3, 3]"
4,0x00000000005ef87f8ca7014309ece7260bbcdaeb,2022-01-10 19:19:47,"[12, 3]","[0.7818320421108522, -0.7818309228245777]","[1.4960439, 1.3195738]","[-0.025127048, -0.025127048]","[-0.0009817483, -0.0009817483]","[4, 3]","[3, 4]","[3, 3]"


In [9]:
cudf_df_select = cudf_df[['recipient','buyAsset-list']]

In [10]:
cudf_df_select.head()

,recipient,buyAsset-list
0,0x000000000004d7463d0f9c77383600bc82d612f5,"[489, 418, 956, 24, 359, 1476, 1017, 714, 503,..."
1,0x0000000000086013192a2c9a506742a43c916e1e,"[4, 12, 150, 4]"
2,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113]"
3,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,"[614, 4]"
4,0x00000000005ef87f8ca7014309ece7260bbcdaeb,"[12, 3]"


In [11]:
print(cudf_df_select.dtypes)

recipient        object
buyAsset-list      list
dtype: object


In [12]:
cudf_df_select_pd  = cudf_df_select.to_pandas()

In [13]:
cudf_df_select_pd.dtypes

recipient        object
buyAsset-list    object
dtype: object

In [14]:
cudf_df_select_pd.head()

,recipient,buyAsset-list
0,0x000000000004d7463d0f9c77383600bc82d612f5,"[489, 418, 956, 24, 359, 1476, 1017, 714, 503,..."
1,0x0000000000086013192a2c9a506742a43c916e1e,"[4, 12, 150, 4]"
2,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113]"
3,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,"[614, 4]"
4,0x00000000005ef87f8ca7014309ece7260bbcdaeb,"[12, 3]"


In [16]:
cudf_df_select_pd['last_element'] = cudf_df_select_pd['buyAsset-list'].apply(lambda x: x[-1])
cudf_df_select_pd['last_last_element'] = cudf_df_select_pd['buyAsset-list'].apply(lambda x: x[-2])


In [17]:
cudf_df_select_pd.head()

,recipient,buyAsset-list,last_element,last_last_element
0,0x000000000004d7463d0f9c77383600bc82d612f5,"[489, 418, 956, 24, 359, 1476, 1017, 714, 503,...",941,388
1,0x0000000000086013192a2c9a506742a43c916e1e,"[4, 12, 150, 4]",4,150
2,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113]",113,5
3,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,"[614, 4]",4,614
4,0x00000000005ef87f8ca7014309ece7260bbcdaeb,"[12, 3]",3,12


In [18]:
cate_data = cudf.read_parquet(f"./preprocessWorkflow/categories/unique.buyAsset.parquet")
cate_data_pd = cate_data.to_pandas()

In [19]:
cate_data_pd = cate_data.to_pandas()

In [20]:
cate_data_pd.head()

,buyAsset,buyAsset_size
3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,126211
4,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,119359
5,0xdac17f958d2ee523a2206206994597c13d831ec7,83574
6,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,25439
7,0x6b175474e89094c44da98b954eedeac495271d0f,21678


In [21]:
cate_data_pd.index

RangeIndex(start=3, stop=1515, step=1)

In [22]:
import numpy as np
import pandas as pd

def generate_random_integers():
    return list(np.random.randint(3, 1515, size=5))

# Apply the custom function to each row and create a new column 'random_integers'
cudf_df_select_pd['random_asset'] = cudf_df_select_pd.apply(lambda row: generate_random_integers(), axis=1)

# Display the resulting DataFrame
cudf_df_select_pd.head()

,recipient,buyAsset-list,last_element,last_last_element,random_asset
0,0x000000000004d7463d0f9c77383600bc82d612f5,"[489, 418, 956, 24, 359, 1476, 1017, 714, 503,...",941,388,"[121, 1323, 1005, 1404, 1410]"
1,0x0000000000086013192a2c9a506742a43c916e1e,"[4, 12, 150, 4]",4,150,"[443, 1508, 516, 1249, 1457]"
2,0x000000000008e4db6a6194c6957df47e30970dc2,"[4, 5, 113]",113,5,"[1384, 516, 1486, 1326, 941]"
3,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,"[614, 4]",4,614,"[853, 626, 141, 211, 366]"
4,0x00000000005ef87f8ca7014309ece7260bbcdaeb,"[12, 3]",3,12,"[832, 1103, 1304, 603, 1469]"
